# Bab 11: Training Deep Neural Networks

### 1. Pendahuluan

Bab 11 membahas berbagai tantangan yang muncul saat melatih *Deep Neural Networks* (DNN) dan menyajikan serangkaian teknik canggih untuk mengatasinya. Meskipun secara teori DNN sangat kuat, dalam praktiknya kita bisa menghadapi berbagai masalah seperti:
* **Vanishing/Exploding Gradients:** Membuat lapisan-lapisan bawah sulit untuk dilatih.
* **Kurangnya data latih** atau biaya pelabelan yang mahal.
* **Pelatihan yang sangat lambat.**
* **Overfitting** karena model memiliki jutaan parameter.

Bab ini akan membahas solusi untuk setiap masalah ini, termasuk inisialisasi bobot yang lebih baik, fungsi aktivasi yang lebih canggih, *Batch Normalization*, penggunaan ulang *layer* dari model yang sudah ada (*transfer learning*), *optimizer* yang lebih cepat, dan teknik regularisasi seperti *dropout*.

---

### 2. Masalah Vanishing/Exploding Gradients

Selama proses *backpropagation*, gradien seringkali menjadi semakin kecil saat algoritma bergerak ke lapisan yang lebih bawah. Akibatnya, bobot koneksi dari lapisan bawah hampir tidak berubah, dan pelatihan tidak pernah konvergen. Ini disebut masalah **vanishing gradients**. Kebalikannya, **exploding gradients**, terjadi ketika gradien menjadi semakin besar, menyebabkan pembaruan bobot yang sangat besar dan membuat model divergen.

#### a. Inisialisasi Glorot dan He
Untuk mengatasi masalah ini, diperlukan inisialisasi bobot acak yang cermat. Strategi seperti **inisialisasi Glorot (atau Xavier)** dan **inisialisasi He** memastikan bahwa varians dari output setiap lapisan sama dengan varians inputnya, yang secara signifikan membantu mencegah gradien menghilang atau meledak.

Secara *default*, Keras menggunakan inisialisasi Glorot dengan distribusi uniform. Saat menggunakan fungsi aktivasi ReLU atau variannya, lebih baik menggunakan inisialisasi He.


In [1]:
import tensorflow as tf
from tensorflow import keras

# Contoh penggunaan inisialisasi He
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

<Dense name=dense, built=False>

### b. Fungsi Aktivasi Nonsaturating
Fungsi aktivasi seperti sigmoid dan tanh memiliki area "saturasi" di mana turunannya mendekati nol. Ketika ini terjadi, gradien berhenti mengalir selama backpropagation. Untuk mengatasinya, digunakan fungsi aktivasi yang tidak mengalami saturasi seperti ReLU dan variannya.
* **Leaky ReLU**: $ \text{LeakyReLU}_{\alpha}(z) = \max(\alpha z, z) $. Memperbaiki masalah "dying ReLUs" dengan mengizinkan sedikit gradien mengalir bahkan untuk input negatif.
* **ELU (Exponential Linear Unit)**: Mengungguli varian ReLU lainnya, tetapi sedikit lebih lambat untuk dihitung.
* **SELU (Scaled ELU)**: Jika arsitektur hanya terdiri dari lapisan-lapisan Dense yang berurutan, SELU dapat membuat jaringan self-normalize, yang seringkali memberikan performa superior.

In [2]:
# Contoh penggunaan Leaky ReLU
model = keras.models.Sequential([
    # [...]
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    # [...]
])

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


---

### 3. Batch Normalization (BN)
BN adalah teknik yang mengatasi masalah vanishing/exploding gradients dengan cara menambahkan operasi normalisasi sebelum atau sesudah fungsi aktivasi di setiap hidden layer. Teknik ini menormalkan output dari lapisan sebelumnya dengan mengurangi rata-rata dan membaginya dengan standar deviasi, lalu menskalakan dan menggesernya.

Manfaat BN:
* Model menjadi jauh lebih tidak sensitif terhadap inisialisasi bobot.
* Memungkinkan penggunaan learning rate yang lebih besar, yang mempercepat pelatihan.
* Bertindak sebagai regularizer, mengurangi kebutuhan teknik regularisasi lain seperti dropout.

In [3]:
# Implementasi Batch Normalization di Keras
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Penempatan layer BN biasanya setelah layer Dense (sebelum aktivasi) atau setelah aktivasi. Keduanya umum digunakan dan hasilnya cenderung serupa.

---

### 4. Gradient Clipping
Teknik ini sangat berguna untuk mencegah exploding gradients, terutama dalam Recurrent Neural Networks (RNN). Caranya adalah dengan membatasi nilai gradien agar tidak melebihi threshold tertentu selama backpropagation.

In [4]:
# Menerapkan Gradient Clipping di Keras
optimizer = keras.optimizers.SGD(clipvalue=1.0) # atau clipnorm=1.0
model.compile(loss="mse", optimizer=optimizer)

---

### 5. Transfer Learning dengan Keras
Daripada melatih DNN dari awal, seringkali jauh lebih cepat dan efektif untuk menggunakan kembali lapisan-lapisan dari jaringan yang sudah dilatih pada dataset besar yang serupa (misalnya, ImageNet). Teknik ini disebut Transfer Learning.

Prosesnya:

1. Bekukan lapisan-lapisan awal (Freeze Layers): Lapisan-lapisan awal dari model pre-trained biasanya mendeteksi fitur-fitur tingkat rendah (garis, tepi, bentuk). Bobotnya diatur agar tidak dapat dilatih (layer.trainable = False).
2. Ganti atau Tambah Lapisan Output: Ganti lapisan output asli dengan lapisan baru yang sesuai dengan tugas Anda.
3. Latih Model: Latih model pada dataset baru Anda. Awalnya, hanya lapisan baru yang dilatih.
4. (Opsional) Fine-Tuning: Setelah beberapa epoch, Anda bisa "mencairkan" (unfreeze) beberapa lapisan atas dari model pre-trained dan melatihnya dengan learning rate yang sangat kecil untuk sedikit menyesuaikannya dengan data baru Anda.

In [5]:
# Contoh Transfer Learning menggunakan Xception pada Fashion MNIST
# (Catatan: Fashion MNIST adalah gambar grayscale, contoh ini disederhanakan)
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Memuat model Xception yang sudah dilatih di ImageNet, tanpa lapisan atasnya
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                   include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(10, activation="softmax")(avg)
model_transfer = keras.Model(inputs=base_model.input, outputs=[output])

# Membekukan bobot dari base model
for layer in base_model.layers:
    layer.trainable = False

# Compile dan latih model dengan lapisan baru
optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
model_transfer.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# history = model_transfer.fit(X_train, y_train, ...) # Perlu preprocessing data agar sesuai

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 74s 1us/step


c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


---

### 6. Faster Optimizers
Mengganti optimizer SGD standar dengan yang lebih canggih dapat secara signifikan mempercepat pelatihan. Beberapa optimizer populer:

* **Momentum optimization**: Mempercepat konvergensi dengan menambahkan "momentum" pada pembaruan bobot.
* **Nesterov Accelerated Gradient (NAG)**: Varian dari momentum yang sedikit lebih cepat.
* **AdaGrad, RMSProp, Adam, dan Nadam**: Optimizer adaptif yang secara otomatis menyesuaikan learning rate untuk setiap parameter, seringkali menjadi pilihan default yang sangat baik. Adam dan Nadam adalah yang paling sering direkomendasikan.

In [6]:
# Menggunakan optimizer Adam di Keras
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

---

### 7. Learning Rate Scheduling
Menyesuaikan learning rate selama pelatihan dapat membantu model konvergen lebih cepat dan mencapai solusi yang lebih baik. Beberapa strategi penjadwalan (scheduling) umum:

* **Power scheduling**: Mengurangi LR secara bertahap di setiap iterasi.
* **Exponential scheduling**: Mengurangi LR sebesar faktor 10 setiap S langkah.
* **Piecewise constant scheduling**: Menggunakan learning rate konstan untuk beberapa epoch, lalu menguranginya.
* **Performance scheduling**: Mengukur validation error dan mengurangi LR ketika error berhenti menurun.
* **1cycle scheduling**: Menaikkan LR dari nilai rendah ke tinggi, lalu menurunkannya kembali selama satu siklus pelatihan.

In [7]:
# Contoh implementasi Performance Scheduling
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# history = model.fit(X_train, y_train, ..., callbacks=[lr_scheduler])

---

### 8. Regularisasi untuk Mencegah Overfitting
* **$\ell_1$ and $\ell_2$ Regularization**: Menambahkan penalti pada loss function berdasarkan nilai absolut ($\ell_1$) atau kuadrat ($\ell_2$) dari bobot model.
* **Dropout**: Pada setiap langkah pelatihan, setiap neuron (termasuk input, tetapi tidak output) memiliki probabilitas untuk "dijatuhkan" (dropped out) untuk sementara. Ini memaksa neuron lain untuk belajar fitur yang lebih robust.
* **Max-Norm Regularization**: Membatasi bobot koneksi masuk agar tidak melebihi nilai maksimum tertentu.

In [8]:
# Implementasi Dropout di Keras
model_dropout = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])